In [39]:
from tensorflow.keras.preprocessing import sequence
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPool1D, Dropout, SimpleRNN, LSTM, Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import sequence
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

In [40]:
df = pd.read_csv('Epileptic Seizure Recognition.csv')

1. Pre-process the raw data.

In [41]:
df = pd.read_csv('Epileptic Seizure Recognition.csv')
print(df.head().T)
df = df.drop('Unnamed', 1)
y_df = df['y']
print(y_df.head().T)
print(y_df.value_counts())

,0,1,2,3,4
Unnamed,X21.V1.791,X15.V1.924,X8.V1.1,X16.V1.60,X20.V1.54
X1,135,386,-32,-105,-9
X2,190,382,-39,-101,-65
X3,229,356,-47,-96,-98
X4,223,331,-37,-92,-102
...,...,...,...,...,...
X175,-127,156,-30,-77,-65
X176,-116,154,-35,-72,-83
X177,-83,143,-35,-69,-89
X178,-51,129,-36,-65,-73


In [46]:
y_binary = pd.Series(np.where(df['y'].values == 1, 1, 0), df['y'].index)
print(y_binary.head().T)
print(y_binary.value_counts())

In [49]:
x_df = df.drop('y', 1)

/var/folders/pg/1pf8c0d10hs7bc4233k0d06h0000gp/T/ipykernel_58147/76009137.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_df = df.drop('y', 1)


2. Training.

In [50]:
test_recall = []
test_f1= []
test_acc = []


for i in range(30):

    under_sampler = RandomUnderSampler(random_state=1)
    X_res, y_res = under_sampler.fit_resample(x_df, y_binary)

    X = []

    for i, row in X_res.iterrows():
        tmp_x = []
        for e in row.tolist():
            tmp_x.append([e])
        X.append(tmp_x)
        
    X = np.array(X)

    model = Sequential()
    model.add(Input(shape=(178, 1)))
    model.add(MaxPool1D(pool_size=2))
    model.add(Conv1D(16, 3, input_shape=(50, 20000), strides=1, padding='causal', activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(Conv1D(8, 3, input_shape=(25, 20000), strides=1, padding='causal', activation='relu'))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    print(model.summary())

    X_train, X_test, y_train, y_test = train_test_split(X, y_res, test_size=0.2, random_state=1234)
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=2000, batch_size=256, verbose=2)

    y_pred1 = model.predict(X_test)
    y_pred = np.argmax(y_pred1, axis=1)

    # Print f1, precision, and recall scores
    test_acc.append((accuracy_score(y_test, y_pred)))
    test_recall.append((recall_score(y_test, y_pred)))
    test_f1.append((f1_score(y_test, y_pred)))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d_6 (MaxPooling  (None, 89, 1)            0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 89, 16)            64        
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 44, 16)           0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 44, 8)             392       
                                                                 
 flatten_3 (Flatten)         (None, 352)               0         
                                                                 
 dense_6 (Dense)             (None, 250)              

2023-02-06 22:17:49.266612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


29/29 - 1s - loss: 6.3107 - accuracy: 0.5677 - val_loss: 2.5066 - val_accuracy: 0.6152 - 794ms/epoch - 27ms/step
Epoch 2/30


2023-02-06 22:17:49.789509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


29/29 - 0s - loss: 1.6637 - accuracy: 0.6959 - val_loss: 1.4436 - val_accuracy: 0.7152 - 294ms/epoch - 10ms/step
Epoch 3/30
29/29 - 0s - loss: 0.8917 - accuracy: 0.7685 - val_loss: 1.1689 - val_accuracy: 0.7435 - 243ms/epoch - 8ms/step
Epoch 4/30
29/29 - 0s - loss: 0.5630 - accuracy: 0.8196 - val_loss: 1.0344 - val_accuracy: 0.7652 - 257ms/epoch - 9ms/step
Epoch 5/30
29/29 - 0s - loss: 0.4217 - accuracy: 0.8533 - val_loss: 0.9864 - val_accuracy: 0.7630 - 239ms/epoch - 8ms/step
Epoch 6/30
29/29 - 0s - loss: 0.3181 - accuracy: 0.8799 - val_loss: 1.0311 - val_accuracy: 0.7924 - 239ms/epoch - 8ms/step
Epoch 7/30
29/29 - 0s - loss: 0.2407 - accuracy: 0.9098 - val_loss: 1.1155 - val_accuracy: 0.8087 - 239ms/epoch - 8ms/step
Epoch 8/30
29/29 - 0s - loss: 0.1868 - accuracy: 0.9307 - val_loss: 0.8337 - val_accuracy: 0.8239 - 235ms/epoch - 8ms/step
Epoch 9/30
29/29 - 0s - loss: 0.1219 - accuracy: 0.9560 - val_loss: 0.9166 - val_accuracy: 0.8402 - 237ms/epoch - 8ms/step
Epoch 10/30
29/29 - 0s - l

2023-02-06 22:17:57.204031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [54]:
test_recall = pd.DataFrame(test_recall, columns=['test_recall'])
test_f1 = pd.DataFrame(test_f1, columns=['test_f1'])
test_acc = pd.DataFrame(test_acc, columns=['test_acc'])

evaluation_result = pd.concat([test_recall, test_f1, test_acc],axis=1)
evaluation_result.to_csv('evaluation_baseline_CNN.csv')

In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()